In [2]:
import pandas as pd 
import numpy as np 

df = pd.read_csv("processed_data/numerical.csv", index_col="EmployeeID")
X = df[df.columns.difference(['Attrition_num'])]
y = df['Attrition_num']

# Low variance elimination

<img src="assets/variance.PNG" width="300" height="300">

Vymaž črty (stĺpce) s nízkou varianciou. Celá myšlienka je taká, že stĺpce s nízkou varianciou majú malú pridanú hodnotu pre predikciu.

Výhody:
- Veľmi ľahká implementácia
- Rýchla implementácia 
- Dá sa aplikovať aj pri algoritmoch učenia bez učiteľa kedy nemáme predikovanú premennú

Nevýhody:
- Ako nastaviť správnu hodnotu variance treshold?
- Problém pri nevyvážených datasetoch kedy stĺpce s nízkou varianciou môžu byť dôležité pre minoritnú triedu
- Je dôležité aby boli dáta na rovnakej škále

In [3]:
# Viac si precitate sem 
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold
data = [[0, 2, 0, 3], [0, 1, 4, 3], [0, 1, 1, 3]]
temp_df=pd.DataFrame(data)
temp_df

,0,1,2,3
0,0,2,0,3
1,0,1,4,3
2,0,1,1,3


In [4]:
temp_df[0].var()

0.0

In [5]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
variance = VarianceThreshold(threshold=.05)

X_scaled = scaler.fit_transform(X)
print(X_scaled.shape)
X_selected = variance.fit_transform(X_scaled)
X_selected.shape

(4382, 13)


(4382, 7)

In [6]:
pd.DataFrame(X_scaled,columns=list(X.columns)).var()

Age                        0.047330
DistanceFromHome           0.083798
Education                  0.065629
JobLevel                   0.076468
MonthlyIncome              0.061627
NumCompaniesWorked         0.077027
PercentSalaryHike          0.068457
StockOptionLevel           0.080731
TotalWorkingYears          0.037886
TrainingTimesLastYear      0.046182
YearsAtCompany             0.023481
YearsSinceLastPromotion    0.046225
YearsWithCurrManager       0.044092
dtype: float64

# Univariate feature selection
Výber tých čŕt, ktoré majú najsilnejší vzťah s predikovanou premennou. Vzťah vyjadríme pomocou štatistického testu. Poznáme rôzne druhy štatistických testov, ktoré vieme použiť v závislosti od typu stĺpca a vlastností stĺpca. 

Výhody:
- Veľmi ľahká implementácia
- Rýchla implementácia 

Nevýhody:
- Je potrebná predikovaná premenná (nevhodné pre unsupervised algoritmi)
- Je pouzitelna pre samostatne premenne (existuju pristupy aj pre kombinovane premenne)
- Je potrebna rovnaka skala dat
- Problem vybrat spravny test
- Ako vybrat spravne k?

V prípade spojiných premenných použijeme ANOVA f-test [pekné vysvetlenie sem](https://datascience.stackexchange.com/questions/74465/how-to-understand-anova-f-for-feature-selection-in-python-sklearn-selectkbest-w)
V pripace kategorických atribútov sa použíje chi2 test [pekné vysvetlenie sem](https://towardsdatascience.com/chi-square-test-for-feature-selection-in-machine-learning-206b1f0b8223)

In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

scaler = MinMaxScaler()
feature_selection = SelectKBest(f_classif, k=4)

X_scaled = scaler.fit_transform(X)
print(X_scaled.shape)
X_selected = feature_selection.fit_transform(X_scaled, y)
X_selected.shape

(4382, 13)


(4382, 4)

# Recursive feature elimination (RFE)
Táto technika používa externý prediktor na to aby určila ako veĺmi sú dôležité črty pre predikciu. Postup algoritmu (veľmi zjednodušene):
- Začneme so všetkými črtami a pomocou externého prediktoru zbehneme klasifikáciu zapíšeme úspešnosť klasifikácie.
- Pozrieme sa na dôležitosť jednotlivých čŕt pri klasifikácii a eleminujeme tú čo najmenej prispieva.
- Proces opakujeme do vtedy pokiaľ sa úspešnosť predikcie zlepšuje

Výhody:
- Veľmi populárne a hlavne úspešné riešenie 

Nevýhody 
- Veľmi pomalé riešenie: je potrebné natrénovať extery klasifikátor púre každú iteráciu
- Je potrebné zvoliť externý klasifikátor, ktorý vie poskynúť dôležitosť čŕt po trénovaní.

In [8]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

print(X.shape)
scaler = MinMaxScaler()
feature_selection = RFE(LogisticRegression())

X_scaled = scaler.fit_transform(X)
feature_selection = feature_selection.fit(X_scaled, y)

print(feature_selection.support_)
print(feature_selection.ranking_)

X_selected = feature_selection.transform(X_scaled)
X_selected.shape

(4382, 13)
[ True False False False False  True False False  True  True False  True
  True]
[1 8 3 7 2 1 5 6 1 1 4 1 1]


(4382, 6)

In [9]:
X.columns

Index(['Age', 'DistanceFromHome', 'Education', 'JobLevel', 'MonthlyIncome',
       'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')